![QuantConnect Logo](https://cdn.quantconnect.com/web/i/icon.png)
<hr>

#### Initializing environment

In [2]:
// We need to load assemblies at the start in their own cell
#load "../Initialize.csx"
// QuantBook C# Research Environment
// For more information see https://www.quantconnect.com/docs/v2/our-platform/research/getting-started
#load "../QuantConnect.csx"

#### Loading historical BTC prices in daily resolution

In [3]:
// Import necessary namespaces
using QuantConnect;
using QuantConnect.Data;
using QuantConnect.Data.Market;
using QuantConnect.Brokerages;
using System;
using System.Linq;

// Initialize QuantBook
var qb = new QuantConnect.Research.QuantBook();

// Set account currency and brokerage model
qb.SetAccountCurrency("USDT");
qb.SetBrokerageModel(BrokerageName.Binance, AccountType.Cash);

// Add a crypto subscription
var btcUsdt = qb.AddCrypto("BTCUSDT", Resolution.Daily, Market.Binance).Symbol;

// Set the date range for historical data
var startDate = new DateTime(2013, 04, 07);
var endDate = new DateTime(2024, 11, 1);

// Request historical data
var btcHistory = qb.History<TradeBar>(btcUsdt, startDate, endDate, Resolution.Daily).ToList();

// Verify data retrieval
Console.WriteLine($"Number of data points retrieved: {btcHistory.Count}");
foreach (var bar in btcHistory.Take(5))
{
    Console.WriteLine($"Date: {bar.EndTime}, Open: {bar.Open}, High: {bar.High}, Low: {bar.Low}, Close: {bar.Close}, Volume: {bar.Volume}");
}
foreach (var bar in btcHistory.Skip(btcHistory.Count-5))
{
    Console.WriteLine($"Date: {bar.EndTime}, Open: {bar.Open}, High: {bar.High}, Low: {bar.Low}, Close: {bar.Close}, Volume: {bar.Volume}");
}

#### Computing Benchmark return

We assume an initial investment of 5000$

In [4]:
// Calculer le rendement cumulatif du benchmark
decimal initialInvestment = 5000;
Console.WriteLine($"Initial Portfolio Value: 5000$: BTC Close {btcHistory.First().Close}$/BTC");
var cumulativeReturn = new List<(DateTime Time, decimal PortfolioValue)>();
decimal runningTotal = initialInvestment;

foreach (var bar in btcHistory)
{
    runningTotal = initialInvestment * (bar.Close / btcHistory.First().Close);
    cumulativeReturn.Add((bar.EndTime, runningTotal));
}

// Afficher quelques rendements
foreach (var point in cumulativeReturn.Take(5))
{
    Console.WriteLine($"Date: {point.Time}, Portfolio Value: {point.PortfolioValue:C}");
}

Console.WriteLine($"Final Portfolio Value: {cumulativeReturn.Last().Time}: {cumulativeReturn.Last().PortfolioValue:C} BTC Close {btcHistory.Last().Close}$/BTC");


#### Identifying peaks for various margins

We want to extract local extrema for various margins and select those that yield a target amount of peaks (around 50 peaks for a handful of target yearly trades)

In [5]:
// Introducing a method to compute peaks from history given a target percentage

public List<(DateTime Time, decimal Price)> GetPeaks(List<TradeBar> data, decimal thresholdPercentage)
{
    var peaks = new List<(DateTime Time, decimal Price)>();
    if (data.Count == 0) return peaks;

    decimal threshold = thresholdPercentage / 100m;

    // On peut choisir de commencer par chercher un pic (lookingForMax = true) ou un creux (lookingForMax = false).
    // Arbitrairement, on commence par chercher un max.
    bool lookingForMax = true;

    // Le dernier pivot connu. On démarre à partir du premier point.
    int pivotIndex = 0;
    decimal pivotPrice = data[0].Close;

    for (int i = 1; i < data.Count; i++)
    {
        var price = data[i].Close;

        if (lookingForMax)
        {
            // On cherche un pic
            if (price > pivotPrice)
            {
                // On monte toujours, on met à jour le pivot (candidat max)
                pivotIndex = i;
                pivotPrice = price;
            }
            else
            {
                // On baisse: calcul de la variation depuis le pivot max potentiel
                decimal drop = (pivotPrice - price) / pivotPrice;
                if (drop > threshold)
                {
                    // On a suffisamment baissé pour valider le pivot comme un pic
                    peaks.Add((data[pivotIndex].EndTime, pivotPrice));
                    // Maintenant on cherche un creux
                    lookingForMax = false;
                    // Le pivot devient le point actuel (qui servira de base pour trouver un creux)
                    pivotIndex = i;
                    pivotPrice = price;
                }
            }
        }
        else
        {
            // On cherche un creux
            if (price < pivotPrice)
            {
                // On descend encore, on met à jour le pivot (candidat min)
                pivotIndex = i;
                pivotPrice = price;
            }
            else
            {
                // On monte: calcul de la variation depuis le pivot min potentiel
                decimal rise = (price - pivotPrice) / pivotPrice;
                if (rise > threshold)
                {
                    // On a suffisamment monté pour valider le pivot comme un creux
                    peaks.Add((data[pivotIndex].EndTime, pivotPrice));
                    // Maintenant on cherche un pic
                    lookingForMax = true;
                    // Le pivot devient le point actuel (qui servira de base pour trouver un pic)
                    pivotIndex = i;
                    pivotPrice = price;
                }
            }
        }
    }

    return peaks;
}


// Computing peak numbers for various target percentages

var peakPercentages = new List<decimal> { 5, 15, 25, 40 };

var peakNumbers = peakPercentages.ToDictionary(
    percentage => percentage,
    percentage => GetPeaks(btcHistory, percentage)
);

// Displaying the peak numbers

foreach (var kvp in peakNumbers)
{
    Console.WriteLine($"Peaks at {kvp.Key}%: {kvp.Value.Count}");
    foreach (var peak in kvp.Value.Take(5))
    {
        Console.WriteLine($"Date: {peak.Time}, Price: {peak.Price:C}");
    }
}

#### Computing ideal strategy and max theoretical return

We now calculate the theoretical maximum return based on the selected peaks, assuming the ideal strategy of buying at local minima and selling at local maxima.


In [6]:
// Assuming 30% threshold is appropriate based on prior analysis
var selectedPeaks = peakNumbers[25];

// Recalculating theoretical maximum return based on the selected peaks, starting with an initial buy
decimal initialInvestment = 5000m;
decimal cash = initialInvestment; // Start with all cash
decimal btcHoldings = 0m;         // Initially no BTC


decimal firstPrice = btcHistory.First().Close; // First day's close price

// Simulate buying at the first day's price
btcHoldings = cash / firstPrice;
cash = 0; // All cash converted to BTC

bool buying = false; // Always start by buying

// Process the peaks
for (int i = 0; i < selectedPeaks.Count; i++)
{
    var peak = selectedPeaks[i];

    if (buying)
    {

        // Buy at the current price using all cash
        Console.WriteLine($"Buying at {peak.Price:C} on {peak.Time}");
        btcHoldings = cash / peak.Price;
        cash = 0; // All cash converted to BTC
    }
    else
    {
        // Sell all BTC at the current price
        Console.WriteLine($"Selling at {peak.Price:C} on {peak.Time}");
        cash = btcHoldings * peak.Price;
        btcHoldings = 0; // All BTC converted to cash
    }

    // Alternate between buying and selling
    buying = !buying;
}

// Final portfolio value is either in cash or converted BTC
decimal finalPortfolioValue = cash + (btcHoldings * btcHistory.Last().Close);
decimal theoreticalReturn = (finalPortfolioValue / initialInvestment) - 1;

// Display the results
Console.WriteLine($"Theoretical Max Return: {theoreticalReturn:P}");
Console.WriteLine($"Number of Trades: {selectedPeaks.Count + 1}"); // Include the initial trade
Console.WriteLine($"Final Portfolio Value: {finalPortfolioValue:C}");


#### Introducing a Peak class for convenience

In [7]:
// Définition d'une classe pour représenter un pic
public class Peak
{
    public DateTime Time { get; set; }
    public decimal Price { get; set; }
    public bool IsMax { get; set; } // True si c'est un sommet (local maximum), False si c'est un creux (local minimum)

    // Fenêtres dynamiques basées sur un pourcentage du pic
    public DateTime StartWindow { get; set; }
    public DateTime EndWindow { get; set; }

    // On pourra ajouter d'autres propriétés plus tard si nécessaire
}

// Supposons que selectedPeaks est actuellement une List<(DateTime Time, decimal Price)>.
// Il faut maintenant créer une nouvelle liste de Peak.

// Logique : On part du principe que les pics sont alternés max/min/max/min ... 
// Sinon, il faudrait une logique de détermination (par ex. comparer le price du pic précédent et suivant).
// Pour simplifier, on admet qu'on sait déjà si on est sur un max ou un min en alternant :
var peaks = new List<Peak>();
bool isMax = true; 
foreach (var p in selectedPeaks)
{
    peaks.Add(new Peak
    {
        Time = p.Time,
        Price = p.Price,
        IsMax = isMax
    });
    isMax = !isMax; // On alterne. À adapter selon la vraie logique.
}

// À ce stade, on a une liste de Peak avec Time, Price, et IsMax définis.
// Les fenêtres dynamiques ne sont pas encore définies.
// Vérification du nombre de max et min et de la structure des pics

int maxCount = peaks.Count(p => p.IsMax);
int minCount = peaks.Count(p => !p.IsMax);

Console.WriteLine($"Max count: {maxCount}, Min count: {minCount}");

//Affichage des premières valeurs pour vérification

foreach (var peak in peaks.Take(5))
{
    Console.WriteLine($"Time: {peak.Time}, Price: {peak.Price:C}, IsMax: {peak.IsMax}");
}




#### Defining dynamic windows around peaks

In [11]:
// Paramètres globaux
decimal windowPercent = 0.20m;
var priceByDate = btcHistory.ToDictionary(x => x.EndTime, x => x.Close);
var orderedDates = btcHistory.Select(x => x.EndTime).OrderBy(d => d).ToList();

// Fonctions auxiliaires unifiées pour le calcul des fenêtres max et min.
// L'idée est de renvoyer un couple (start, end) cohérent, avec start < end si possible.

(DateTime start, DateTime end) ComputeDynamicWindow(
    Peak peak,
    decimal windowPercent,
    List<DateTime> orderedDates,
    Dictionary<DateTime, decimal> priceByDate)
{
    // Pour un max, le seuil est "peakPrice * (1 - windowPercent)"
    // Pour un min, le seuil est "peakPrice * (1 + windowPercent)"
    // On va unifier la logique dans une seule fonction, en tenant compte de peak.IsMax
    decimal threshold = peak.IsMax 
                       ? peak.Price * (1 - windowPercent)
                       : peak.Price * (1 + windowPercent);

    // Cherche la date de début (startWindow)
    // - Si c'est un max, on remonte dans le temps jusqu'à price < threshold
    // - Si c'est un min, on remonte dans le temps jusqu'à price > threshold
    // On veut la date *juste après* ce franchissement pour éviter start == peak
    DateTime startWindow = FindWindowStart(peak, threshold, orderedDates, priceByDate);

    // Cherche la date de fin (endWindow)
    // - Si c'est un max, on avance dans le temps jusqu'à price < threshold
    // - Si c'est un min, on avance dans le temps jusqu'à price > threshold
    // On veut la date *juste avant* ce franchissement pour éviter end == peak
    DateTime endWindow = FindWindowEnd(peak, threshold, orderedDates, priceByDate);

    // Assurer start < end. Si jamais start == end, on essaie de corriger en décalant end d’un jour
    if (startWindow >= endWindow)
    {
        // On peut choisir d'étendre endWindow d’un cran si possible.
        // Sinon, on laisse tel quel et on signalera un invalid window plus tard.
        var idxEnd = orderedDates.BinarySearch(endWindow);
        if (idxEnd >= 0 && idxEnd < orderedDates.Count - 1)
        {
            endWindow = orderedDates[idxEnd + 1];
        }
    }

    return (startWindow, endWindow);
}

// Pour un max, on remonte dans le temps et on s'arrête quand price < threshold, 
// puis on retourne la date juste APRÈS ce franchissement.
DateTime FindWindowStart(Peak peak, decimal threshold, List<DateTime> orderedDates, Dictionary<DateTime, decimal> priceByDate)
{
    var index = orderedDates.BinarySearch(peak.Time);
    if (index < 0) index = ~index;

    for (int i = index - 1; i >= 0; i--)
    {
        var date = orderedDates[i];
        var price = priceByDate[date];
        bool crossing = peak.IsMax ? price < threshold : price > threshold;
        if (crossing)
        {
            // On prend la date i+1 pour éviter la fenêtre nulle
            return (i + 1 < orderedDates.Count) ? orderedDates[i + 1] : orderedDates[i];
        }
    }
    // Si on n'a rien trouvé, on prend le début
    return orderedDates.First();
}

// Pour un max, on avance dans le temps et on s'arrête quand price < threshold, 
// puis on retourne la date juste AVANT ce franchissement.
DateTime FindWindowEnd(Peak peak, decimal threshold, List<DateTime> orderedDates, Dictionary<DateTime, decimal> priceByDate)
{
    var index = orderedDates.BinarySearch(peak.Time);
    if (index < 0) index = ~index;

    for (int i = index + 1; i < orderedDates.Count; i++)
    {
        var date = orderedDates[i];
        var price = priceByDate[date];
        bool crossing = peak.IsMax ? price < threshold : price > threshold;
        if (crossing)
        {
            // On prend la date i-1 pour éviter la fenêtre nulle
            return (i - 1 >= 0) ? orderedDates[i - 1] : orderedDates[i];
        }
    }
    // Si on n'a rien trouvé, on prend la fin
    return orderedDates.Last();
}

// Maintenant on calcule les fenêtres pour chaque pic en appelant ComputeDynamicWindow
foreach (var peak in peaks)
{
    (DateTime s, DateTime e) = ComputeDynamicWindow(peak, windowPercent, orderedDates, priceByDate);
    peak.StartWindow = s;
    peak.EndWindow   = e;
}


#### Introducing Indicators

We start by introducing a range of ADX and MACD indicator with various parameters.
Our simple starting strategy is defined by:

            var macdHistogram = _macd.Current.Value - _macd.Signal.Current.Value;

            // Détermination des signaux MACD
            var isMacdBullish = macdHistogram > 0;    // Signal haussier si l'histogramme est positif
            var isMacdBearish = macdHistogram < 0;    // Signal baissier si l'histogramme est négatif

            // Récupération de la valeur actuelle de l'ADX
            var adxValue = _adx.Current.Value;

            // Conditions d'entrée en position longue
            if (adxValue >= AdxHigh && isMacdBullish)
            {
                // Si le portefeuille n'est pas déjà investi
                if (!Portfolio.Invested)
                {
                    // Investit 100% du capital disponible dans le symbole
                    SetHoldings(_symbol, 1);
                    // Debug($"Acheté {_symbol} au prix de {currentPrice}");
                }
            }
            // Conditions de sortie de position
            else if (adxValue < AdxLow && isMacdBearish)
            {
                // Si le portefeuille est investi
                if (Portfolio.Invested)
                {
                    // Liquide la position sur le symbole
                    Liquidate(_symbol);
                    // Debug($"Vendu {_symbol} au prix de {currentPrice}");
                }
            }

Accordingly, we want to adjust adx parameters so that they are compatible with our peaks, the adjust MACD accordingly.


#### Generating ADX with Various Periods

We generate a range of ADX indicators with different periods. 
For each indicator, we will observe how its value changes in the window around the detected peaks.


In [12]:
//Extending period for ADX warming
// Extend historical data range to ensure indicators have enough data to warm up
var extendedStartDate = startDate.AddYears(-1); // Add one extra year for warming up
var extendedBtcHistory = qb.History<TradeBar>(btcUsdt, extendedStartDate, endDate, Resolution.Daily).ToList();



// Define ADX periods to test
var adxPeriods = new List<int> { 5, 10, 15, 20, 25, 30, 50, 100 };

// Dictionary to store ADX values for each period
var adxIndicators = adxPeriods.ToDictionary(
    period => period,
    period =>
    {
        var adx = qb.ADX(btcUsdt, period, Resolution.Daily);
        var adxValues = new List<(DateTime Time, decimal Value)>();

        foreach (var bar in extendedBtcHistory)
        {
            adx.Update(bar);
            if (adx.IsReady)
            {
                adxValues.Add((bar.EndTime, adx.Current.Value));
            }
        }

        return adxValues;
    }
);

// Display ADX values for a few periods
Console.WriteLine("ADX Values for Different Periods:");
foreach (var period in adxPeriods)
{
    Console.WriteLine($"ADX Period: {period}");
    foreach (var value in adxIndicators[period].Skip(365).Take(5))
    {
        Console.WriteLine($"Date: {value.Time}, ADX: {value.Value:F2}");
    }
    Console.WriteLine("---");
}


#### Correlation Between ADX Changes and Peaks

We analyze how ADX values behave in a defined window around each peak.
The goal is to identify periods where ADX changes align well with the peaks.


In [13]:
// Define window size (in days) around each peak for analysis
int windowSize = 30; // Days before and after each peak

// Analyze ADX behavior around each peak for each period
var adxPeakAnalysis = adxPeriods.ToDictionary(
    period => period,
    period =>
    {
        var adxValues = adxIndicators[period];
        return selectedPeaks.Select(peak =>
        {
            var window = adxValues
                .Where(x => x.Time >= peak.Time.AddDays(-windowSize) && x.Time <= peak.Time.AddDays(windowSize))
                .ToList();

            return new
            {
                Peak = peak,
                Period = period,
                Window = window,
                AvgADX = window.Any() ? window.Average(x => x.Value) : 0,
                MaxADX = window.Any() ? window.Max(x => x.Value) : 0,
                MinADX = window.Any() ? window.Min(x => x.Value) : 0
            };
        }).ToList();
    }
);

// Display results for one period
int examplePeriod = 25; // Example ADX period
Console.WriteLine($"ADX Analysis for Period {examplePeriod}:");
foreach (var analysis in adxPeakAnalysis[examplePeriod].Take(20))
{
    Console.WriteLine($"Date: {analysis.Peak.Time}, Price: {analysis.Peak.Price}, AvgADX: {analysis.AvgADX:F2}, MaxADX: {analysis.MaxADX:F2}, MinADX: {analysis.MinADX:F2}");
}


#### Dynamic Window analysis

In [14]:
var periodToAnalyze = 25; // exemple
var adxData = adxIndicators[periodToAnalyze]; // List<(DateTime Time, decimal Value)>
var adxByDate = adxData.ToDictionary(x => x.Time, x => x.Value);

// Fonction auxiliaire pour moyenne ADX sur une période
decimal AverageAdxInInterval(DateTime start, DateTime end)
{
    var subset = adxData.Where(x => x.Time >= start && x.Time <= end).Select(x => x.Value).ToList();
    if (subset.Count == 0) return 0;
    return subset.Average();
}

// Analysons la moyenne d'ADX dans les fenêtres vs juste avant
// Par exemple, on prend la même durée que la fenêtre, juste avant le StartWindow, pour comparer.
foreach (var peak in peaks)
{
    var windowDuration = (peak.EndWindow - peak.StartWindow).TotalDays;
    if (windowDuration <= 0) continue;

    var adxInWindow = AverageAdxInInterval(peak.StartWindow, peak.EndWindow);

    // Intervalle juste avant la fenêtre, de même durée
    var preWindowStart = peak.StartWindow.AddDays(-windowDuration);
    var preAdx = AverageAdxInInterval(preWindowStart, peak.StartWindow);

    Console.WriteLine($"{peak.Time}: {(peak.IsMax ? "Max" : "Min")} Price={peak.Price}, ADX in Window={adxInWindow:F2}, ADX before Window={preAdx:F2}");
}

// Une fois cet affichage produit, on pourrait calculer des stats globales:
// Moyenne des ADX dans les fenêtres, moyennes hors fenêtres, etc.


In [15]:
int windowDays = 10; // fenêtre de ±10 jours autour du pic
decimal adxThreshold = 25m; // seuil ADX à tester
var periodsToTest = new List<int> { 5, 10, 15, 20, 25, 30, 50, 100 };

public class AdxPeakAnalysisResult
{
    public int AdxPeriod { get; set; }
    public decimal Threshold { get; set; }
    public int TotalPeaks { get; set; }
    public int PeaksWithCrossing { get; set; }
    public double AverageDaysDifference { get; set; }
    public List<double> Differences = new List<double>();
}

// Fonction pour analyser l'alignement des franchissements ADX avec les pics
public AdxPeakAnalysisResult AnalyzeAdxAroundPeaks(
    List<(DateTime Time, decimal Price)> peaks,
    List<(DateTime Time, decimal Value)> adxValues,
    decimal adxThreshold,
    int windowDays)
{
    var result = new AdxPeakAnalysisResult
    {
        Threshold = adxThreshold
    };

    // Pour faciliter l’accès, on va transformer adxValues en dictionnaire par date
    // Mais comme ce sont des données journalières alignées, on pourra faire une recherche par proximité
    // On suppose ici que adxValues est une liste déjà ordonnée par date
    var adxByDate = adxValues.ToDictionary(x => x.Time, x => x.Value);
    var adxDates = adxValues.Select(x => x.Time).ToList();

    result.TotalPeaks = peaks.Count;

    foreach (var peak in peaks)
    {
        // Déterminer l'intervalle de dates
        var startDate = peak.Time.AddDays(-windowDays);
        var endDate = peak.Time.AddDays(windowDays);

        // Extraire les points ADX dans la fenêtre
        var windowData = adxValues.Where(x => x.Time >= startDate && x.Time <= endDate).OrderBy(x => x.Time).ToList();

        // On cherche un franchissement du seuil ADX dans la fenêtre
        // Un franchissement se produit lorsqu’on passe d'un ADX < threshold à ADX >= threshold ou l’inverse
        // On va détecter le premier franchissement le plus proche du pic, et mesurer la différence en jours

        double? bestDiff = null; // meilleur écart en jours (valeur absolue la plus faible)
        for (int i = 1; i < windowData.Count; i++)
        {
            var prev = windowData[i - 1];
            var curr = windowData[i];

            bool crossUp = prev.Value < adxThreshold && curr.Value >= adxThreshold;
            bool crossDown = prev.Value > adxThreshold && curr.Value <= adxThreshold;

            if (crossUp || crossDown)
            {
                // On suppose le franchissement entre prev.Time et curr.Time
                // Pour simplifier, on prend la date du curr comme date du franchissement
                var diff = (curr.Time - peak.Time).TotalDays;
                var absDiff = Math.Abs(diff);

                // On cherche le franchissement le plus proche du pic
                if (bestDiff == null || absDiff < bestDiff)
                {
                    bestDiff = absDiff;
                }
            }
        }

        if (bestDiff.HasValue)
        {
            result.PeaksWithCrossing++;
            result.Differences.Add(bestDiff.Value);
        }
    }

    if (result.PeaksWithCrossing > 0)
    {
        result.AverageDaysDifference = result.Differences.Average();
    }

    return result;
}

// On applique maintenant l'analyse sur plusieurs périodes
foreach (var period in periodsToTest)
{
    var adxData = adxIndicators[period]; // récupéré précédemment
    var analysis = AnalyzeAdxAroundPeaks(selectedPeaks, adxData, adxThreshold, windowDays);
    analysis.AdxPeriod = period;

    Console.WriteLine($"ADX Period: {period}, Threshold: {adxThreshold}, Window: ±{windowDays} days");
    Console.WriteLine($"Total Peaks: {analysis.TotalPeaks}, Peaks w/ Crossing: {analysis.PeaksWithCrossing} ({(double)analysis.PeaksWithCrossing / analysis.TotalPeaks:P})");
    if (analysis.PeaksWithCrossing > 0)
    {
        Console.WriteLine($"Average Days Difference (abs): {analysis.AverageDaysDifference:F2} days");
    }
    Console.WriteLine("----------------------------------------------------");
}


#### Analyse des min et max de l'ADX dans fenêtre dynamique

In [16]:
// Nous partons du principe que 'peaks' et 'adxIndicators' sont déjà définis.
// Ici, on va se concentrer sur une seule période d'ADX, par exemple 25, mais on peut facilement faire une boucle.

// Choix de la période d'ADX
int periodToAnalyze = 25;
var adxData = adxIndicators[periodToAnalyze]; // List<(DateTime Time, decimal Value)>
var adxByDate = adxData.ToDictionary(x => x.Time, x => x.Value);

// Fonction auxiliaire pour extraire les stats d'ADX (min, max, moyenne) sur un intervalle
public class AdxStats
{
    public decimal Min { get; set; }
    public decimal Max { get; set; }
    public decimal Avg { get; set; }
    public int Count { get; set; }
}

public AdxStats GetAdxStats(DateTime start, DateTime end, List<(DateTime Time, decimal Value)> adxData)
{
    var subset = adxData.Where(x => x.Time >= start && x.Time <= end).Select(x => x.Value).ToList();
    if (subset.Count == 0)
    {
        return new AdxStats { Min = 0, Max = 0, Avg = 0, Count = 0 };
    }
    return new AdxStats
    {
        Min = subset.Min(),
        Max = subset.Max(),
        Avg = subset.Average(),
        Count = subset.Count
    };
}

// Pour chaque pic, on va calculer :
// - ADX dans la fenêtre dynamique (window)
// - ADX juste avant la fenêtre (pre-window) sur la même durée, si c'est un max, pour comparer la montée
// - ADX juste après la fenêtre (post-window) sur la même durée, si c'est un min, pour comparer la remontée
//
// Cela permet de voir si, par exemple, sur un max, l'ADX dans la fenêtre (autour du pic) est plus élevé que juste avant, 
// ce qui indiquerait une tendance ascendante forte se dissipant ensuite.

// On collecte les résultats dans des listes séparées pour les max et les min, afin de faire des stats globales ensuite.
var maxPeakResults = new List<(DateTime Time, decimal Price, AdxStats WindowStats, AdxStats PreStats)>();
var minPeakResults = new List<(DateTime Time, decimal Price, AdxStats WindowStats, AdxStats PostStats)>();

foreach (var peak in peaks)
{
    var windowDuration = (peak.EndWindow - peak.StartWindow).TotalDays;
    if (windowDuration <= 0) continue; // Fenêtre invalide ou trop petite

    // Stats ADX dans la fenêtre
    var windowStats = GetAdxStats(peak.StartWindow, peak.EndWindow, adxData);

    // On va comparer : 
    // - Si c'est un max, on regarde la période juste avant la fenêtre, sur la même durée
    // - Si c'est un min, on regarde la période juste après la fenêtre, sur la même durée
    if (peak.IsMax)
    {
        var preWindowStart = peak.StartWindow.AddDays(-windowDuration);
        var preStats = GetAdxStats(preWindowStart, peak.StartWindow, adxData);
        maxPeakResults.Add((peak.Time, peak.Price, windowStats, preStats));
    }
    else
    {
        var postWindowEnd = peak.EndWindow.AddDays(windowDuration);
        var postStats = GetAdxStats(peak.EndWindow, postWindowEnd, adxData);
        minPeakResults.Add((peak.Time, peak.Price, windowStats, postStats));
    }
}

// Maintenant, on calcule des stats globales pour voir si la fenêtre autour du pic est typiquement plus ou moins élevée en ADX que la période de comparaison.
decimal AvgDiffMax = 0;
decimal AvgDiffMin = 0;

if (maxPeakResults.Count > 0)
{
    // Pour les max : on regarde la différence entre l'ADX moyen dans la fenêtre et l'ADX moyen avant.
    var diffs = maxPeakResults
        .Where(x => x.WindowStats.Count > 0 && x.PreStats.Count > 0)
        .Select(x => x.WindowStats.Avg - x.PreStats.Avg)
        .ToList();

    if (diffs.Count > 0)
    {
        AvgDiffMax = diffs.Average();
    }

    Console.WriteLine($"Max Peaks Count: {maxPeakResults.Count}");
    Console.WriteLine($"Average difference (WindowAvg - PreWindowAvg) for Max: {AvgDiffMax:F2} ADX");
}

if (minPeakResults.Count > 0)
{
    // Pour les min : on regarde la différence entre l'ADX moyen dans la fenêtre et l'ADX moyen après.
    var diffs = minPeakResults
        .Where(x => x.WindowStats.Count > 0 && x.PostStats.Count > 0)
        .Select(x => x.WindowStats.Avg - x.PostStats.Avg)
        .ToList();

    if (diffs.Count > 0)
    {
        AvgDiffMin = diffs.Average();
    }

    Console.WriteLine($"Min Peaks Count: {minPeakResults.Count}");
    Console.WriteLine($"Average difference (WindowAvg - PostWindowAvg) for Min: {AvgDiffMin:F2} ADX");
}

// On peut aussi sortir quelques exemples concrets
Console.WriteLine("\nExamples for Max peaks:");
foreach (var r in maxPeakResults.Take(5))
{
    Console.WriteLine($"{r.Time} (Max at {r.Price:C}): Window ADX Avg={r.WindowStats.Avg:F2}, PreWindow ADX Avg={r.PreStats.Avg:F2}");
}

Console.WriteLine("\nExamples for Min peaks:");
foreach (var r in minPeakResults.Take(5))
{
    Console.WriteLine($"{r.Time} (Min at {r.Price:C}): Window ADX Avg={r.WindowStats.Avg:F2}, PostWindow ADX Avg={r.PostStats.Avg:F2}");
}


In [19]:
// Choix d'une période d'ADX
int periodToAnalyze = 25;
var adxData = adxIndicators[periodToAnalyze]; // List<(DateTime Time, decimal Value)>
var adxByDate = adxData.ToDictionary(x => x.Time, x => x.Value);

// Nous allons construire 4 distributions : 
// 1. ADX autour des MAX (dans la fenêtre dynamique)
// 2. ADX autour des MAX (hors de la fenêtre dynamique, mais dans la zone "autour du pic")
// 3. ADX autour des MIN (dans la fenêtre dynamique)
// 4. ADX autour des MIN (hors de la fenêtre dynamique, mais dans la zone "autour du pic")

// Remarque : "Hors fenêtre" ne signifie pas tout le reste de l'historique, 
// mais seulement la partie autour du pic (par ex. ± X jours OU ±X% du prix).
// Ici on va se limiter à la fenêtre dynamique qu'on a déjà définie (StartWindow, EndWindow) 
// et prendre, disons, une "fenêtre élargie" plus large pour avoir une zone "autour" 
// (ex. StartWindow - X jours jusqu'à EndWindow + X jours). 
// Puis on compare la portion "strictement dans la fenêtre" VS "hors fenêtre" dans cette zone élargie.

double extraDays = 20; // Par exemple, on prend 20 jours de plus avant/après pour définir la zone élargie

var adxMaxWindow   = new List<decimal>();
var adxMaxOutside  = new List<decimal>();
var adxMinWindow   = new List<decimal>();
var adxMinOutside  = new List<decimal>();

foreach (var peak in peaks)
{
    // Fenêtre dynamique
    DateTime startWin = peak.StartWindow;
    DateTime endWin   = peak.EndWindow;
    if (endWin <= startWin) 
        continue; // On ignore ce pic s'il a une fenêtre invalide

    // Fenêtre élargie = on ajoute +/- extraDays autour de la fenêtre 
    // pour obtenir une zone plus grande de comparaison.
    DateTime bigStart = startWin.AddDays(-extraDays);
    DateTime bigEnd   = endWin.AddDays(extraDays);
    
    var subset = adxData.Where(x => x.Time >= bigStart && x.Time <= bigEnd).ToList();
    // Subset = toutes les valeurs ADX dans la zone élargie

    foreach (var adxPoint in subset)
    {
        bool inWindow = (adxPoint.Time >= startWin && adxPoint.Time <= endWin);
        if (peak.IsMax)
        {
            if (inWindow) adxMaxWindow.Add(adxPoint.Value);
            else          adxMaxOutside.Add(adxPoint.Value);
        }
        else
        {
            if (inWindow) adxMinWindow.Add(adxPoint.Value);
            else          adxMinOutside.Add(adxPoint.Value);
        }
    }
}

// Maintenant on calcule les stats globales sur ces 4 distributions
// Pour un résumé "propre" : min, max, moyenne, médiane, (quartiles)

static (decimal Min, decimal Max, decimal Mean, decimal Median, int Count) GetDistributionStats(List<decimal> values)
{
    if (!values.Any()) return (0, 0, 0, 0, 0);
    values.Sort();
    decimal min = values.First();
    decimal max = values.Last();
    decimal mean = values.Average();
    decimal median = (values.Count % 2 == 1)
        ? values[values.Count / 2]
        : (values[values.Count / 2 - 1] + values[values.Count / 2]) / 2;
    return (min, max, mean, median, values.Count);
}

var statsMaxWindow   = GetDistributionStats(adxMaxWindow);
var statsMaxOutside  = GetDistributionStats(adxMaxOutside);
var statsMinWindow   = GetDistributionStats(adxMinWindow);
var statsMinOutside  = GetDistributionStats(adxMinOutside);

// Affichage final
Console.WriteLine($"ADX Period = {periodToAnalyze}");
Console.WriteLine($"ExtraDays for comparison = {extraDays}\n");

Console.WriteLine("=== MAX Peaks ===");
Console.WriteLine($"Window  : Count={statsMaxWindow.Count}, " +
                  $"Min={statsMaxWindow.Min:F2}, Max={statsMaxWindow.Max:F2}, Mean={statsMaxWindow.Mean:F2}, Median={statsMaxWindow.Median:F2}");
Console.WriteLine($"Outside : Count={statsMaxOutside.Count}, " +
                  $"Min={statsMaxOutside.Min:F2}, Max={statsMaxOutside.Max:F2}, Mean={statsMaxOutside.Mean:F2}, Median={statsMaxOutside.Median:F2}");

Console.WriteLine("\n=== MIN Peaks ===");
Console.WriteLine($"Window  : Count={statsMinWindow.Count}, " +
                  $"Min={statsMinWindow.Min:F2}, Max={statsMinWindow.Max:F2}, Mean={statsMinWindow.Mean:F2}, Median={statsMinWindow.Median:F2}");
Console.WriteLine($"Outside : Count={statsMinOutside.Count}, " +
                  $"Min={statsMinOutside.Min:F2}, Max={statsMinOutside.Max:F2}, Mean={statsMinOutside.Mean:F2}, Median={statsMinOutside.Median:F2}");
